In [26]:
import os
import logging
import json
import psycopg2
import pandas as pd

def connect_to_database():
    # Define your database connection parameters
    db_params = {
        "host": "localhost",
        "database": "blockchain",
        "user": "postgres",
        "password": "auth2020"
    }

    try:
        # Connect to the database
        conn = psycopg2.connect(**db_params)
        print("Connected to the database successfully!")
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")
        return None

In [27]:
import psycopg2

def get_most_recent_transaction_id(conn):
    try:
        with conn.cursor() as cursor:
            # Query to retrieve the most recent transaction_exec_id
            get_transaction_id_query = """
                SELECT transaction_begin_redelegate_id
                FROM transaction_header_begin_redelegate
                ORDER BY transaction_begin_redelegate_id DESC
                LIMIT 1;
            """
            cursor.execute(get_transaction_id_query)
            result = cursor.fetchone()
            if result:
                transaction_id = result[0]
                print(f"Most recent transaction_begin_redelegate_id: {transaction_id}")
                return transaction_id
            else:
                print("No transaction_exec_id found.")
                return None
    except psycopg2.Error as e:
        print(f"Error retrieving transaction_exec_id: {e}")
        return None


In [28]:
import psycopg2

def insert_transaction_detail(conn, transaction_id, delegator_address, validator_source_address, validator_target_address, denom_value, amount_value):
    try:
        table_name = 'transaction_detail_begin_deregulate'
        with conn.cursor() as cursor:
            # Construct the insert query for the current message
            msg_insert_query = f"""
                INSERT INTO {table_name} (transaction_id, delegator_address, validator_source_address, validator_target_address, denom, amount)
                VALUES ('{transaction_id}', '{delegator_address}', '{validator_source_address}', '{validator_target_address}', '{denom_value}', {amount_value});
            """
            cursor.execute(msg_insert_query)
            conn.commit()
            print("Data inserted successfully!")
    except psycopg2.Error as e:
        print(f"Error inserting data: {e}")

# Example usage: Replace with actual values
try:
    conn = psycopg2.connect(db_params)
    # Replace with actual values
    delegator_address = json_data["tx"]["body"]["messages"][0]["delegator_address"]
    validator_source_address = json_data["tx"]["body"]["messages"][0]["validator_src_address"]
    validator_target_address = json_data["tx"]["body"]["messages"][0]["validator_dst_address"]
    amount_value = json_data["tx"]["body"]["messages"][0]["amount"][0]["amount"]
    denom_value = json_data["tx"]["body"]["messages"][0]["amount"][0]["denom"]
    transaction_id = get_most_recent_transaction_id(conn)  # Retrieve the transaction ID
    insert_transaction_detail(conn, transaction_id, delegator_address, validator_source_address, validator_target_address, denom_value, amount_value)
    conn.close()
except psycopg2.Error as e:
    print(f"Error connecting to the database: {e}")


NameError: name 'db_params' is not defined